<a href="https://colab.research.google.com/github/Ini-design/GenAi/blob/main/Copy_of_Untitled7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
GROQ_API_KEY = ""

In [ ]:
# Cell 2 - Imports and basic setup
import os
import getpass
from io import BytesIO
from PIL import Image
import torch
from diffusers import StableDiffusionPipeline
import gradio as gr

# Groq client (uses the groq python package)
from groq import Client

# Use the API key set in the previous cell
client = Client(api_key=GROQ_API_KEY)

In [ ]:
# Cell 3 - Helper: use Groq LLM to expand / refine user prompt into a detailed image prompt
def expand_prompt_with_groq(user_prompt: str, model: str = "llama-3.1") -> str:
    """
    Sends a short user prompt to Groq LLM and requests a high-quality detailed prompt
    optimized for image-generation (Stable Diffusion / diffusion models).
    """
    # Minimal system/user style messages -- adjust to your Groq model choice
    messages = [
        {"role": "system", "content": "You are an assistant that rewrites short user image requests into detailed, photorealistic prompts for image-generation models."},
        {"role": "user", "content": f"Rewrite and expand this into a rich image prompt for a diffusion model:\n\n{user_prompt}\n\nInclude camera, lighting, mood, color palette, style keywords, and useful negative prompts separated by 'NEGATIVE:'"}
    ]
    # SDK usage pattern may vary by groq package version; many examples use `chat.completions.create`.
    # This call returns a completion; the exact attribute names may differ slightly across versions.
    resp = client.chat.completions.create(model=model, messages=messages, max_tokens=512, temperature=0.8)
    # Try to access common response structures
    text = None
    try:
        text = resp.choices[0].message["content"]
    except Exception:
        try:
            text = resp.choices[0].text
        except Exception:
            text = str(resp)
    return text.strip()

In [ ]:
# Cell 4 - Load Stable Diffusion (adjust model id if you prefer another)
device = "cuda" if torch.cuda.is_available() else "cpu"
# Use float16 when on CUDA for memory savings
dtype = torch.float16 if device == "cuda" else torch.float32

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=dtype,
    safety_checker=None,   # optionally integrate a safety checker
    revision="fp16" if device == "cuda" else None
)
pipe = pipe.to(device)
pipe.enable_attention_slicing()  # reduce memory if needed

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
# Cell 5 - Image generation function (prompt can include "NEGATIVE:" block)
def generate_image_from_prompt(prompt: str, num_inference_steps: int = 28, guidance_scale: float = 7.5, seed: int | None = None):
    # If user provided negative prompts in the same text, split them out
    neg = None
    if "NEGATIVE:" in prompt.upper():
        parts = prompt.split("NEGATIVE:")
        prompt_pos = parts[0].strip()
        neg = parts[1].strip()
    else:
        prompt_pos = prompt

    generator = torch.Generator(device=device)
    if seed is not None:
        generator = generator.manual_seed(seed)
    else:
        generator = None

    out = pipe(
        prompt=prompt_pos,
        negative_prompt=neg,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=generator,
        height=512,
        width=512
    )
    image = out.images[0]
    return image

In [ ]:
# Cell 6 - Gradio UI wiring
def run_pipeline(user_text, model_choice="llama-3.1", steps=28, guidance=7.5, seed=None):
    expanded = expand_prompt_with_groq(user_text, model=model_choice)
    img = generate_image_from_prompt(expanded, num_inference_steps=int(steps), guidance_scale=float(guidance), seed=(int(seed) if seed not in (None, "") else None))
    buf = BytesIO()
    img.save(buf, format="PNG")
    buf.seek(0)
    return expanded, buf

with gr.Blocks() as demo:
    gr.Markdown("## Text → Groq LLM prompt expansion → Stable Diffusion image generator")
    with gr.Row():
        with gr.Column(scale=2):
            prompt_in = gr.Textbox(label="User prompt", placeholder="a cozy cottage in a snowy forest, at dawn")
            model_sel = gr.Dropdown(choices=["llama-3.1","llama-3.3","mixtral-8x7b"], value="llama-3.1", label="Groq LLM model")
            steps = gr.Slider(10, 50, value=28, step=1, label="Inference steps")
            guidance = gr.Slider(1.0, 20.0, value=7.5, step=0.1, label="Guidance scale")
            seed_in = gr.Number(value=None, label="Seed (optional)")
            gen_btn = gr.Button("Generate")
        with gr.Column(scale=3):
            expanded_out = gr.Textbox(label="Expanded prompt from Groq LLM")
            image_out = gr.Image(label="Generated image")

    gen_btn.click(fn=run_pipeline, inputs=[prompt_in, model_sel, steps, guidance, seed_in], outputs=[expanded_out, image_out])

demo.launch(share=False)